In [1]:
import os

import pandas as pd
import numpy as np
import tensorboard.notebook as tbnb

import flatiron.core.tools as fict
import flatiron.core.dataset as ficd
import flatiron.core.logging as ficl
import flatiron.tf.models.unet as fimu
from flatiron.core.pipeline import *

2025-02-17 21:02:38.179709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739826158.196702   18337 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739826158.201841   18337 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 21:02:38.219724: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# CPU only
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
uri = 'askdgashdjkash'
config = f'''
engine: tensorflow
model:
    input_width: 208
    input_height: 208
    input_channels: 3
    classes: 1
    filters: 16
    layers: 7
    # activation:
    # batch_norm:
    # output_activation:
    # kernel_initializer:
    # attention_gates:
    # attention_activation_1:
    # attention_activation_2:
    # attention_kernel_size:
    # attention_strides:
    # attention_padding:
    # attention_kernel_initializer:
dataset:
    source: /mnt/storage/projects/unet001/dset001/p-unet001_s-dset001_d-glom_v001
    ext_regex: npy
    labels: [3]
    label_axis: -1
    limit: 1000
    test_size: 0.2
    reshape: False
optimizer:
    name: sgd
compile:
    loss: jaccard_loss
    metrics:
        - jaccard
        - dice
callbacks:
    project: unet001
    root: /mnt/storage/projects
train:
    batch_size: 16
logger:
    timezone: 'America/Los_Angeles'
    slack_url: 'https://hooks.slack.com/services/{uri}'
    slack_channel: dev
    slack_methods: []
'''
# pipe = fimu.UNetPipeline \
#     .from_string(config) \
#     .build() \
#     .compile() \
#     .train_test_split() \
#     .load() \
#     .train()

pipe = fimu.UNetPipeline \
    .from_string(config) \
    .run()

2025-02-17 21:02:42.098725: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-02-17 21:02:42.098782: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: fae1a180f6f6
2025-02-17 21:02:42.098797: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: fae1a180f6f6
2025-02-17 21:02:42.098958: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.144.3
2025-02-17 21:02:42.099000: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.144.3
2025-02-17 21:02:42.099014: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.144.3


May not total to 100% - Loading Dataset Files:   0%|          | 0/8 [00:00<?, ?it/s]

May not total to 100% - Loading Dataset Files:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 2/30


/home/ubuntu/pdm/envs/pdm-kVbOHlCT-dev-3.10/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(None, 208, 208, 3))
  warnings.warn(msg)


 5/63 ━━━━━━━━━━━━━━━━━━━━ 30:34 32s/step - dice: 0.0000e+00 - jaccard: 0.0000e+00 - loss: 0.4995